# OrthodoxAI - RAG

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

This notebook is dedicated to prototyping and testing **Retrieval-Augmented Generation (RAG)** methods within the **OrthodoxAI** framework.

It serves as a sandbox to:

- Evaluate different libraries and methods for building vector stores  
- Benchmark the performance and timing of the retrieval process  
- Isolate and debug components before integrating them into the main agent system

This notebook is focused solely on the retrieval layer of RAG, without involving agent roles or generation logic at this stage.

## Libraries

In [11]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

## Chroma Vectorstore

### OpenAI

In [10]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma

In [3]:
JSON_DIR = "../src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/"

loader = DirectoryLoader(
    JSON_DIR,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

In [4]:
documents[-1]

Document(metadata={'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/3165_ΑΠΑΝΤΗΣΕΙΣ_ΑΠΟΡΙΩΝ_ΑΝΩΤ_ΚΑΤΗΧΗΤΙΚΟΥ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json', 'seq_num': 1}, page_content="Και πηγαίνουμε πιο κάτω. Ποια είναι η στάση των ανθρώπων μπροστά στο θάνατο; Τι θα μπορούσαμε να πούμε πάνω σε αυτό; Ξέρετε ότι σήμερα οι άνθρωποι στέκονται μπροστά στο θάνατο, να πω τη φράση του Αποστόλου Παύλου; Ως άθεοι εν κόσμω ελπίδα μη έχοντες. Ξέρετε τι σημαίνει αυτό; Αγνοούν την ανάσταση των νεκρών. Ως άθεοι εν κόσμω, ελπίδα μη έχοντες. Τι ελπίδα; Αναστάσεως των νεκρών. Τι κρίμα! Λέει ο Απόστολος Παύλος στους Θεσσαλονικείς , Δεν θέλω να λυπόσαστε όπως και οι λοιποί οι μη έχοντες ελπίδα, αυτοί που δεν έχουν ελπίδα αναστάσεως των νεκρών. Παιδιά, η στάση των ανθρώπων μπροστά στο θάνατο, των ποιών ανθρώπων; Των σωστών, των Χριστιανών, είναι ότι ο Χριστός ανεστήθη και εφόσον ανεστήθη ο Χριστός, θα αναστηθούν και οι νεκροί. Και ο Χριστός είναι ο πρωτότ

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [6]:
chunks[0].page_content

"Πρέπει να ευχαριστήσουμε τον Άγιο Τριαδικό Θεό μας, παιδιά , που μας αξίωσε πάλι να επανέλθουμε εις τον τόπον αυτόν, να ακουστεί ο Θείος του Λόγος. . Στην Αρχαία Εκκλησία η Βάπτισις εγίνεται εις μεγάλην ηλικίαν. Επειδή τώρα υπάρχει ο νηπιο-νηπιοβαπτισμός, η κατήχηση, όπως καταλαβαίνετε, γίνεται-γίνεται η κατήχηση, γίνεται μετά το βάπτισμά μας. Γι' αυτό το λόγο είναι μια αληθινή ευλογία αν κάποτε καταλάβουμε ότι πρέπει να συμπληρώσουμε την κατήχησή μας μετά το βάπτισμά μας, όταν μεγαλώσουμε. Δεν είναι ούτε πολυτέλεια η κατήχηση, ούτε κάτι το έκτακτο ή το, αν θέλετε, το προαιρετικό. Η κατήχηση είναι αναγκαιοτάτη. Γι' αυτό το λόγο, άμα συνειδητο-συνειδητοποιήσομε ότι πρέπει να αποκτήσωμε τη γνώση του περιεχομένου της πίστεώς μας, τότε αναμφισβήτητα θα τρέχουμε όχι στα παιδικά μας και εφηβικά μας χρόνια να συμπληρώνουμε τη γνώση μας, την πνευματική, τη θεολογική, τη χριστιανική, αλλά σε ολόκληρη τη ζωή μας. Οι μαθηταί του Χριστού, μέχρι που πέθαναν, ελέγοντο μαθηταί, που σημαίνει είχανε"

In [7]:
len(chunks)

8574

In [12]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", chunk_size=10) # nomic-embed-text-v1 is god 

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="vectorstore/chroma_db",
    collection_name="athanasios-muthilinaios",
)
vectordb.persist()

/tmp/ipykernel_9623/4027366378.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [8]:
del vectordb

In [13]:
# Re-load your persisted DB
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory="vectorstore/chroma_db",
    collection_name="athanasios-muthilinaios",
)

In [16]:
query = "What did Athanasios Mytilenaios say about humidity?"
results = vectordb.similarity_search_with_score(query, k=5)

# Inspect docs and their distances
for i, (doc, distance) in enumerate(results, start=1):
    # If you prefer a “similarity score” in [0,1], invert the distance:
    similarity_score = 1 - distance

    print(f"--- Result {i} ---")
    print(f"Similarity (higher is better): {similarity_score:.4f}")
    print("Excerpt:", doc.page_content[:300].replace("\n", " "), "…")
    print("Metadata:", doc.metadata)
    print()

--- Result 1 ---
Similarity (higher is better): -0.3112
Excerpt: Χριστός στους ακροατές του, «όταν, λέγει, βραδιάσει», δηλαδή η δύση εε, γίνει εσπέρα, «λέγετε ευδεία», δηλαδή καλός καιρός. «Πυράζει γαρ ο ουρανός, διότι ο ουρανός είναι κόκκινος». Το λέμε και εμείς σήμερα. Όταν η δύσης είναι κόκκινη, αύριο θα έχουμε καλό καιρό. «Και πρωί σήμερον χειμών πυράζει γαρ  …
Metadata: {'seq_num': 1, 'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0009_19-12-82_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json'}

--- Result 2 ---
Similarity (higher is better): -0.3153
Excerpt: είναι το μέτρημα των ανθρωπίνων διακινήσεων. Είναι εκείνο το οποίο λέγει η σοφία του Σολομώντος. Καιρός παντί πράσιν. Πόσο σοφό αυτό; Αν το διαβάσετε, θα δείτε και να λέγει εκεί καιρός του σιγάν και καιρός του λαλείν. Είναι ο καιρός να μιλήσεις, αλλά και ο καιρός να σιωπήσεις. Ο καιρός να γελάσεις κ …
Metadata: {'seq_num': 1, 'source': '/mnt/c/Users/user/Desktop/Orth

### Ollama

In [7]:
from langchain_community.document_loaders import JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
from tqdm.auto import tqdm

# Load & split your transcripts
dir_path = "../src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/"
loader = DirectoryLoader(
    dir_path,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

chunks = splitter.split_documents(documents)
print(f"Total chunks to embed: {len(chunks)}")

# Initialize Chroma DB with Ollama embeddings
persist_dir = "vectorstore/chroma_db"
embeddings = OllamaEmbeddings(
    model="nomic-embed-text-v1",
    base_url="http://127.0.0.1:11435",
)
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_dir,
    collection_name="athanasios-muthlinaios",
)

# Embed & add each chunk with tqdm progress bar
for i, chunk in enumerate(tqdm(chunks, desc="Embedding chunks"), start=1):
    chunk_id = f"{chunk.metadata.get('source', 'doc')}_chunk_{i}"
    chunk.metadata["chunk_id"] = chunk_id
    vectordb.add_documents([chunk], ids=[chunk_id])
    if i % 100 == 0:
        vectordb.persist()
        tqdm.write(f"Persisted {i} chunks…")

# final flush
vectordb.persist()
print(f"✅ All {len(chunks)} chunks embedded via nomic-embed-text-v1 and saved.")

Total chunks to embed: 8574


Embedding chunks:   0%|          | 0/8574 [00:00<?, ?it/s]

ConnectError: [Errno 111] Connection refused